In [ ]:
VOCABULARY = {
    'adverb': [
        'slightly',
        'greatly',
        'smoothly',
        'sharply',
        'slowly',
        'quickly',
        'lightly',
        'significantly',
        'softly',
        'harshly',
        'gradually',
        'immediately',
    ],
    'direction': {
        'X': {
            'left': [
                'left',
                'leftward',
                'to the left'
            ],
            
            'right': [
                'right',
                'rightward',
                'to the right',
            ],
        },
        'Y': {
            'forward': [
                'forward',
                'ahead',
                'to the front',
            ],
            'backward': [
                'backward',
                'behind',
                'to the back',
            ],
        },
        'Z': {
            'up': [
                'up',
                'upward',
                'above',
            ],
            'down': [
                'down',
                'downward',
                'below',
            ],
        },
    },
}

CARTESIAN_AXES = set(VOCABULARY['direction'].keys())
CARTESIAN_DIRECTIONS = {direction for axes_directions in VOCABULARY['direction'].values() for direction in axes_directions.keys()}
CARTESIAN_DIRECTION_TO_AXIS = {direction: axis for axis, axis_directions in VOCABULARY['direction'].items() for direction in axis_directions.keys()}
DIRECTION_PART_TO_AXIS = {direction_part: axis for axis, axis_directions in VOCABULARY['direction'].items() for direction_parts in axis_directions.values() for direction_part in direction_parts}
DIRECTION_PART_TO_CARTESIAN_DIRECTION = {direction_part: direction for axis_directions in VOCABULARY['direction'].values() for direction, direction_parts in axis_directions.items() for direction_part in direction_parts}

import random

def choose_random_part(vocabulary, vocabulary_category):
    possible_options = vocabulary[vocabulary_category]
    if isinstance(possible_options, list):
        return random.choice(possible_options)
    return choose_random_part(possible_options, random.choice(list(possible_options.keys())))

def generate_phrase_parts(phrase_structure):
    return [choose_random_part(VOCABULARY, vocabulary_category) if vocabulary_category in VOCABULARY else vocabulary_category for vocabulary_category in phrase_structure]

def extract_cartesian_directions(phrase_parts):
    return [DIRECTION_PART_TO_CARTESIAN_DIRECTION[part] for part in phrase_parts if part in DIRECTION_PART_TO_CARTESIAN_DIRECTION]

def not_dumb_composite_direction(phrase_parts):
    extracted_cartesian_directions = extract_cartesian_directions(phrase_parts)
    axes = [CARTESIAN_DIRECTION_TO_AXIS[direction] for direction in extracted_cartesian_directions]
    return len(set(axes)) == len(axes)

import csv

def generate_corpus(corpus_path, number_of_users, number_of_each_phrase_type, phrase_types):
    corpus = []

    for user_id in range(1, number_of_users + 1):
        cartesian_directions = list(CARTESIAN_DIRECTIONS)
        random.shuffle(cartesian_directions)
        trial_number = 1
        for direction in cartesian_directions[:6]:
            phrase_text = direction
            adverb = ''
            first_direction = DIRECTION_PART_TO_CARTESIAN_DIRECTION[direction]
            second_direction = ''
            corpus.append([user_id, trial_number, phrase_text, adverb, first_direction, second_direction])
            trial_number += 1

        remaining_trials = []
        for phrase_structure in phrase_types[1:]:
            for _ in range(number_of_each_phrase_type):
                while True:
                    phrase_parts = generate_phrase_parts(phrase_structure)
                    cartesian_directions = extract_cartesian_directions(phrase_parts)

                    if not_dumb_composite_direction(phrase_parts):
                        break

                phrase_text = ' '.join(phrase_parts)
                adverb = next((word for word in phrase_parts if word in VOCABULARY['adverb']), '')
                first_direction = cartesian_directions[0]
                second_direction = cartesian_directions[1] if len(cartesian_directions) > 1 else ''
                remaining_trials.append([
                    user_id,
                    trial_number,
                    phrase_text,
                    adverb,
                    first_direction,
                    second_direction,
                ])

        random.shuffle(remaining_trials)
        for trial in remaining_trials:
            trial[1] = trial_number
            trial_number += 1

        corpus.extend(remaining_trials)

    if corpus_path:
        with open(corpus_path, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([
                'user_id',
                'trial_number',
                'phrase_text',
                'adverb',
                'first_cartesian_direction',
                'second_cartesian_direction',
            ])
            writer.writerows(corpus)
    else:
        return corpus

PHRASE_TYPES = [
    ('direction',),
    ('adverb', 'direction',),
    ('direction', 'adverb',),
    ('direction', 'and', 'direction',),
    ('direction', 'and', 'direction',),
    ('adverb', 'direction', 'and', 'direction',),
    ('direction', 'and', 'direction', 'adverb', ),
]

NUMBER_OF_USERS = 12
NUMBER_OF_EACH_PHRASE_TYPE = 6

# import time

# CORPUS_PATH = f'./corpus_{time.time()}.csv'
# corpus = generate_corpus(CORPUS_PATH, NUMBER_OF_USERS, NUMBER_OF_EACH_PHRASE_TYPE, PHRASE_TYPES)

import pandas as pd

corpus = pd.read_csv('./corpus_1734574556.229892.csv')

corpus

In [ ]:
import preamble
from robot import Robot
from console_command_thread import ConsoleCommandThread
from timer import Timer
from vocalizer import Vocalizer
from virtual_dynamics import SimpleVirtualDynamics
from phrase_trial_data import PhraseTrialData
import numpy as np

USER_ID = 1

def get_phrase_trial(corpus, user_id, trial_number):
    result = corpus[(corpus['user_id'] == user_id) & (corpus['trial_number'] == trial_number)]
    return result.iloc[0] if not result.empty else None

NUM_OF_STAGES = 6
stage = 0

def next_stage():
    global stage
    stage = (stage + 1) % NUM_OF_STAGES

current_trial_number = 1

HOME_POSE = np.array([-0.3660424012449587, 0.6117162981347448, 0.9079274150612482, -0.005400995019868388, -0.0019643877311394183, -2.354619210713495,])

with Robot(
    '169.254.9.43',
    translational_force_deadband=6.0,
    rotational_force_deadband=0.5,
    init_pose=HOME_POSE
) as r, ConsoleCommandThread(['', 'restart', 'current']) as c:

    current_phrase_trial_data = None
    save_current_phrase_trial = False

    vocalizer = Vocalizer(0)

    AXES = Robot.TRANSLATION

    init_point = r.get_pose(AXES)

    timer = Timer()

    vd = SimpleVirtualDynamics(
        M=10,
        B=25,
        K=0,
    )

    while c.is_alive():
        period_start = r.control.initPeriod()

        if stage == 0: # wait for start signal + keep robot still + wait for conductor to verbalize utterance + say current trial number if needed
            if c.poll_command(''):
                next_stage()
                c.reset()
            if c.poll_command('current'):
                phrase_text = get_phrase_trial(corpus, USER_ID, current_trial_number)['phrase_text']
                vocalizer.utter(f'{USER_ID}, {current_trial_number}, {phrase_text}')
                print(f'{USER_ID}, {current_trial_number}, {phrase_text}')
                c.reset()
            elif c.poll_command('exit'):
                break
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
        elif stage == 1: # initiate phrase trial
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
            phrase_trial = get_phrase_trial(corpus, USER_ID, current_trial_number)
            current_phrase_trial_data = PhraseTrialData(
                USER_ID, 
                current_trial_number, 
                phrase_trial['phrase_text'], 
                '' if pd.isna(phrase_trial['adverb']) else phrase_trial['adverb'], 
                phrase_trial['first_cartesian_direction'], 
                '' if pd.isna(phrase_trial['second_cartesian_direction']) else phrase_trial['second_cartesian_direction']
            )
            timer.reset()
            next_stage()
            c.reset()
        elif stage == 2: # robot can move + phrase trial data recording + check for signal
            time = timer.t()

            dt = timer.dt()

            position = r.get_pose(AXES)

            velocity = r.get_velocity(AXES)

            force = r.get_force(AXES)

            current_phrase_trial_data.append(
                time,
                dt,
                position,
                velocity,
                force,
                0,
            )

            vd.apply_force(force, dt)

            r.set_velocity(vd.get_velocity(), AXES, acceleration=10)

            if c.poll_command(''):
                save_current_phrase_trial = True
                next_stage()
                c.reset()
            elif c.poll_command('restart'):
                current_trial_number -= 1
                next_stage()
                c.reset()
            elif c.poll_command('exit'):
                break
        elif stage == 3:
            vd.v = 0.0
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
            if save_current_phrase_trial:
                current_phrase_trial_data.save(current_trial_number, 'lang2force_trial_data')
                save_current_phrase_trial = False
            current_trial_number += 1
            next_stage()
            c.reset()
        elif stage == 4:
            position = r.get_pose(AXES)
            position_delta = init_point - position
            position_delta_magnitude = np.linalg.norm(position_delta)
            position_delta_normalized = position_delta / position_delta_magnitude if position_delta_magnitude != 0 else 0
            return_velocity = 0.2
            r.set_velocity(return_velocity*position_delta_normalized, AXES, acceleration=2)
            if position_delta_magnitude < 0.001:
                r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
                next_stage()
        elif stage == 5:
            r.set_velocity(Robot.get_axes(Robot.zeroed_translation_rotation(), AXES), AXES, acceleration=10)
            c.reset()
            next_stage()

        r.control.waitPeriod(period_start)

print(current_trial_number)